In [2]:
import tensorflow as tf
import musdb
import numpy as np
import functools
#tf.enable_eager_execution()

In [3]:
db = musdb.DB('/Users/igor/src/ml/mss/data/musdb18')
track = db.load_mus_tracks()[0]
track.rate
track.name
# cast to float32 and tensor
tensor = tf.convert_to_tensor(track.audio.astype(np.float32))
# grab the first channel only
tensor = tf.transpose(tensor)[0]
y_tensor = tf.convert_to_tensor(track.stems[0].astype(np.float32))



/anaconda3/envs/mss/lib/python3.7/site-packages/musdb/__init__.py:86: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.setup = yaml.load(f)


In [4]:

# Plot a section of the waveform.
start = track.rate * 10
tensor = tensor[start:start+1*track.rate]
y_tensor=y_tensor[start:start+1*track.rate]

#plt.plot(tensor.numpy())
#plt.show()

NameError: name 'signal' is not defined

In [10]:
signals = tf.reshape(tensor, [1, -1])
y_signals = tf.reshape(y_tensor, [1, -1])

frame_length = 1024
frame_step = 512
fft_length = 1024
stfts = tf.signal.stft(signals, 
    frame_length=frame_length, 
    frame_step=frame_step,
    fft_length=fft_length)
Y_stfts = tf.signal.stft(y_signals, 
    frame_length=frame_length, 
    frame_step=frame_step,
    fft_length=fft_length)
magnitude_spectrograms = tf.abs(stfts)
#plt.imshow(magnitude_spectrograms)
stfts

<tf.Tensor 'stft_2/rfft:0' shape=(1, 85, 513) dtype=complex64>

In [11]:
array = magnitude_spectrograms.numpy().astype(np.float)[0]
plt.imshow(np.swapaxes(array,0,1))
np.swapaxes(array,0,1)



AttributeError: 'Tensor' object has no attribute 'numpy'

In [12]:
recon = tf.signal.inverse_stft(
    stfts=stfts,
    frame_length=frame_length,
    frame_step=frame_step,
    # forward_window_fn
    window_fn=tf.signal.inverse_stft_window_fn(
        frame_step=frame_step,
        forward_window_fn=functools.partial(tf.signal.hann_window, periodic=True),
    )
)

recon

<tf.Tensor 'inverse_stft/overlap_and_add/strided_slice_4:0' shape=(1, 44032) dtype=float32>

In [13]:
plt.plot(recon.numpy().flatten())


AttributeError: 'Tensor' object has no attribute 'numpy'

In [15]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()

model.add(Flatten(input_shape=[1, 85, 513]))
num_layers = 3
layer_size = 256
for _ in range(num_layers):
    model.add(Dense(layer_size, activation='relu'))
#    model.add(Dropout(dropout_amount))
model.add(Dense(1024, activation='softmax'))
# Your code above (Lab 1)
model.compile('adam', 'mean_squared_error') 
model.fit(stfts, Y_stfts)

y_pred_stfts = model.predict(stfts)
model.evaluate(stfts, y_stfts)
scores

ValueError: A target array with shape (1, 171, 513) was passed for an output of shape (None, 1024) while using as loss `mean_squared_error`. This loss expects targets to have the same shape as the output.